In [73]:
from collections import defaultdict
from gensim import corpora
from gensim import models
from gensim.models import Phrases

from pprint import pprint
from mongo_client import MongoDBClient


In [74]:
mdb = MongoDBClient()
df = mdb.get_all_messages()
df.loc[df.text == "", 'text'] = df.loc[df.text == ""]['subject']
df.loc[df.text.isna(), 'text'] = df[df.text.isna()]['subject']
emails = df['text'].tolist()
print(emails[0])

download form 26as two ways e conomic imes sat nov twenty-seven two thousand and twenty-one 11 48 ist itr guide download form 26as two ways individual download form 26as either new income tax portal internet banking facility follow steps download form 26as form 26as important filing itr form 26as become potent tool hands tax authorities important taxpayer know information available cost inflation index fy 2021 22 used ltcg calculation notified income tax department time income tax returns filing next year cii number helpful ascertain long term capital gains liable pay taxes per notification cii fy 2021 22 shall come effect april one two thousand and twenty-two applicable assessment year 2022 23 subsequent assessment years itr filing documents need fy 2020 21 receiving form 16 one must check pan mentioned discrepancy must bring employer s notice employer rectify mistakes form 16 issue revised form itr form applicable fy 2020 21 important ensure income tax return filed using correct itr 

In [75]:
# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(emails)):
    emails[idx] = tokenizer.tokenize(emails[idx])  # Split into words.
    
    # Remove numbers, but not words that contain numbers.
emails = [[token for token in email if not token.isnumeric()] for email in emails]

# Remove words that are only one character.
emails = [[token for token in email if len(token) > 3] for email in emails]
print(emails[0])

['download', 'form', '26as', 'ways', 'conomic', 'imes', 'twenty', 'seven', 'thousand', 'twenty', 'guide', 'download', 'form', '26as', 'ways', 'individual', 'download', 'form', '26as', 'either', 'income', 'portal', 'internet', 'banking', 'facility', 'follow', 'steps', 'download', 'form', '26as', 'form', '26as', 'important', 'filing', 'form', '26as', 'become', 'potent', 'tool', 'hands', 'authorities', 'important', 'taxpayer', 'know', 'information', 'available', 'cost', 'inflation', 'index', 'used', 'ltcg', 'calculation', 'notified', 'income', 'department', 'time', 'income', 'returns', 'filing', 'next', 'year', 'number', 'helpful', 'ascertain', 'long', 'term', 'capital', 'gains', 'liable', 'taxes', 'notification', 'shall', 'come', 'effect', 'april', 'thousand', 'twenty', 'applicable', 'assessment', 'year', 'subsequent', 'assessment', 'years', 'filing', 'documents', 'need', 'receiving', 'form', 'must', 'check', 'mentioned', 'discrepancy', 'must', 'bring', 'employer', 'notice', 'employer', 

In [ ]:
bigram = Phrases(emails, min_count=20)
for idx in range(len(emails)):
    for token in bigram[emails[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            emails[idx].append(token)


In [77]:
len(emails[0])

327

In [78]:
from gensim.corpora import Dictionary
# Create a dictionary representation of the documents.
dictionary = Dictionary(emails)
print(dictionary)

Dictionary(45507 unique tokens: ['26as', 'account', 'address', 'address_book', 'advance']...)


In [79]:
# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=50, no_above=0.5)
len(dictionary)

2541

In [80]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(email) for email in emails]

In [81]:
len(corpus)

2886

In [82]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
len(corpus_tfidf)

2886

In [83]:
from gensim.models import LdaModel

# Set training parameters.
num_topics = 8
chunksize = 2000
passes = 100
iterations = 1000
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus_tfidf,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [84]:
top_topics = model.top_topics(corpus)
print(top_topics[0])

from pprint import pprint
pprint(top_topics)

avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)


[([(0.027096428, 'daily'),
   (0.02599042, 'daily_pick'),
   (0.025428696, 'newsletter_towards'),
   (0.021920618, 'newsletter'),
   (0.021671992, 'pick'),
   (0.015243409, 'browser'),
   (0.013146426, 'monday_thursday'),
   (0.013122994, 'browser_sent'),
   (0.013078136, 'world_examples'),
   (0.013078136, 'daily_ritual'),
   (0.013078136, 'techniques_delivered'),
   (0.013078136, 'research_tutorials'),
   (0.013078136, 'hands_real'),
   (0.013042894, 'ritual'),
   (0.013018338, 'newsletter_view'),
   (0.011465291, 'delivered'),
   (0.011427883, 'cutting_edge'),
   (0.011397906, 'monday'),
   (0.011287219, 'view_browser'),
   (0.011259536, 'latest_picks')],
  -0.4700205476723028),
 ([(0.025354039, 'fund'),
   (0.020407723, 'nippon'),
   (0.01920494, 'mutual'),
   (0.014748404, 'mutual_fund'),
   (0.014044175, 'nippon_india'),
   (0.012730791, 'message'),
   (0.012481086, 'scheme'),
   (0.010976629, 'investments'),
   (0.01054679, 'click'),
   (0.010037002, 'view_message'),
   (0.01001

In [108]:
top_topics = model.top_topics(corpus)
topic_list = list()
for idx, topic in enumerate(top_topics):
    topic_words = topic[0]
    topic = {
        "topic": "topic_" + str(idx),
        "words": list()
    }
    for word_prob in topic_words[1:10]:
        (prob, word) = word_prob
        topic["words"].append(word)
    topic_list.append(topic)
    
pprint(topic_list)

[{'topic': 'topic_0',
  'words': ['daily_pick',
            'newsletter_towards',
            'newsletter',
            'pick',
            'browser',
            'monday_thursday',
            'browser_sent',
            'world_examples',
            'daily_ritual']},
 {'topic': 'topic_1',
  'words': ['nippon',
            'mutual',
            'mutual_fund',
            'nippon_india',
            'message',
            'scheme',
            'investments',
            'click',
            'view_message']},
 {'topic': 'topic_2',
  'words': ['price',
            'markets',
            'nifty',
            'crore',
            'bank',
            'investors',
            'please',
            'newsletter',
            'said']},
 {'topic': 'topic_3',
  'words': ['python',
            'machine',
            'machine_learning',
            'using',
            'data_science',
            'ninety',
            'onezero',
            'talk',
            'better']},
 {'topic': 'topic_4',
  'w

In [85]:
print(corpus)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [86]:
print(dictionary)

Dictionary(2541 unique tokens: ['account', 'address', 'address_book', 'advance', 'advertise']...)


In [87]:
corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 2),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 2),
 (16, 1),
 (17, 1),
 (18, 2),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 4),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 2),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 2),
 (40, 12),
 (41, 2),
 (42, 1),
 (43, 1),
 (44, 1),
 (45, 2),
 (46, 2),
 (47, 1),
 (48, 1),
 (49, 3),
 (50, 2),
 (51, 1),
 (52, 8),
 (53, 1),
 (54, 2),
 (55, 1),
 (56, 1),
 (57, 1),
 (58, 2),
 (59, 2),
 (60, 2),
 (61, 1),
 (62, 1),
 (63, 1),
 (64, 1),
 (65, 1),
 (66, 1),
 (67, 1),
 (68, 1),
 (69, 1),
 (70, 1),
 (71, 3),
 (72, 2),
 (73, 1),
 (74, 1),
 (75, 1),
 (76, 1),
 (77, 1),
 (78, 3),
 (79, 1),
 (80, 2),
 (81, 3),
 (82, 2),
 (83, 1),
 (84, 2),
 (85, 3),
 (86, 1),
 (87, 1),
 (88, 1),
 (89, 1),
 (90, 1),
 (91, 1),
 (92, 1),
 (93, 1),
 (94, 1),
 (95, 1),
 (96, 1),
 (97, 3),
 (98, 1),
 (99, 1),
 (100, 3)

In [89]:
email_topic_probs = model[corpus_tfidf]

In [90]:
for email_topic_prob in email_topic_probs:
    print(email_topic_prob)

[(5, 0.391005), (7, 0.59770775)]
[(0, 0.03640388), (1, 0.016289664), (2, 0.021946022), (3, 0.012013284), (4, 0.017853184), (5, 0.8037722), (6, 0.043508396), (7, 0.04821336)]
[(0, 0.4681522), (5, 0.51634747)]
[(0, 0.03640388), (1, 0.016289664), (2, 0.021946022), (3, 0.012013284), (4, 0.017853184), (5, 0.8037722), (6, 0.043508396), (7, 0.04821336)]
[(0, 0.03640388), (1, 0.016289664), (2, 0.021946022), (3, 0.012013284), (4, 0.017853184), (5, 0.8037722), (6, 0.043508396), (7, 0.04821336)]
[(5, 0.9889517)]
[(5, 0.23988387), (7, 0.7513048)]
[(5, 0.48318964), (7, 0.5092958)]
[(5, 0.43009135), (7, 0.56198984)]
[(5, 0.6122887), (7, 0.37815937)]
[(5, 0.47056085), (6, 0.5159262)]
[(0, 0.03640388), (1, 0.016289664), (2, 0.021946022), (3, 0.012013284), (4, 0.017853184), (5, 0.8037722), (6, 0.043508396), (7, 0.04821336)]
[(5, 0.28944626), (7, 0.70320934)]
[(5, 0.9854211)]
[(5, 0.6701425), (7, 0.32362103)]
[(5, 0.9884986)]
[(5, 0.2308049), (7, 0.7604238)]
[(5, 0.7177927), (7, 0.27527627)]
[(5, 0.9842